In [1]:
!pip install -q tflite_runtime

import numpy as np
import pandas as pd

import os
import json
import tensorflow as tf
import matplotlib.pyplot as plt
import tflite_runtime.interpreter as tflite
import tqdm

from tensorflow.keras.layers import Conv1D, TimeDistributed,MaxPooling2D,Flatten,LSTM,Dense,GlobalMaxPooling2D,Concatenate, Dropout,Bidirectional
from tensorflow.keras import Sequential,Model,Input
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tensorflow.keras.optimizers import Adam



tf.__version__

'2.11.0'

In [2]:
train = pd.read_csv('/kaggle/input/asl-signs/train.csv')

print(train.head())

k = open('/kaggle/input/asl-signs/sign_to_prediction_index_map.json')
sign_to_prediction_index_map = json.load(k)
prediction_to_sign_map = {}
for i, j in sign_to_prediction_index_map.items():
    prediction_to_sign_map[j]=i



ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)#.fillna(0)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)
def load_relevant_data_subset_train(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns).fillna(0)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

                                            path  participant_id  sequence_id  \
0  train_landmark_files/26734/1000035562.parquet           26734   1000035562   
1  train_landmark_files/28656/1000106739.parquet           28656   1000106739   
2   train_landmark_files/16069/100015657.parquet           16069    100015657   
3  train_landmark_files/25571/1000210073.parquet           25571   1000210073   
4  train_landmark_files/62590/1000240708.parquet           62590   1000240708   

    sign  
0   blow  
1   wait  
2  cloud  
3   bird  
4   owie  


type
face          468
pose           33
left_hand      21
right_hand     21
Name: type, dtype: int64

In [3]:
try:
    X = np.load('/kaggle/input/asl-signs-naidu/X.npy')
    y = np.load('/kaggle/input/asl-signs-naidu/y.npy')
    X_train = X[:89753,:,:]
    y_train = y[:89753]
    X_cv = X[89753:,:,:]
    y_cv = y[89753:]
    
except:
    files =train['path'].values
    X = np.zeros((len(train),543,3))
    y = np.zeros((len(train)))
    for i , fname in tqdm.tqdm(enumerate(files)):
        arr = load_relevant_data_subset_train('/kaggle/input/asl-signs/'+fname)
        X[i,:,:] = np.mean(arr,axis=0)
        y[i] = sign_to_prediction_index_map[train.loc[i,'sign']]
    y = y.astype('int32')
    np.save('X.npy',X)
    np.save('y.npy',y)
    X_train = X[:89753,:,:]
    y_train = y[:89753]
    X_cv = X[89753:,:,:]
    y_cv = y[89753:]
    
    


In [4]:
def conv1D_lstm(inputs,filters,kernel_size,lstm_nodes,num_lstm,bi=False):
    for i,j in enumerate(filters):
        inputs = Conv1D(filters=j,kernel_size=kernel_size[i])(inputs) 
        
    if bi:
        if len(lstm_nodes)>1:
            for i in lstm_nodes[:-1]:
                inputs = Bidirectional(LSTM(i,return_sequences=True))(inputs)
            inputs = Bidirectional(LSTM(lstm_nodes[-1]))(inputs)    
            return inputs
        else:
            inputs = Bidirectional(LSTM(lstm_nodes[0]))(inputs)
            return inputs
    else:
        if len(lstm_nodes)>1:
            for i in lstm_nodes[:-1]:
                inputs = LSTM(i,return_sequences=True)(inputs)
            inputs = LSTM(lstm_nodes[-1])(inputs)    
            return inputs
        else:
            inputs = LSTM(lstm_nodes[0])(inputs)
            return inputs
        
            
    

In [5]:
def nn_model():
    inputs = tf.keras.Input(shape=(543,3),name='inputty')
    face = inputs[:,:468,:]
    l_hand = inputs[:,468:468+21,:]
    pose = inputs[:,468+21:468+21+33,:]
    r_hand = inputs[:,468+21+33:468+21+33+21,:]
    
    face = conv1D_lstm(face,[16,32,64],[5,3,3],[128],1,True)
    l_hand = conv1D_lstm(l_hand,[16,32,64],[5,3,3],[128],1,True)
    pose = conv1D_lstm(pose,[16,32,64],[5,3,3],[128],1,True)
    r_hand = conv1D_lstm(r_hand,[16,32,64],[5,3,3],[128],1,True)
    
    
    #face = Dense(units=128)(face)
    #l_hand = Dense(units=128)(l_hand)
    #pose = Dense(units=128)(pose)
    #r_hand = Dense(units=128)(r_hand)
    

    
    
    X = Concatenate()([face,l_hand,pose,r_hand])
    X = Dropout(0.2)(X)
    outputs = Dense(units=250,activation='softmax',name='outputty')(X)
    model = Model(inputs=inputs, outputs = outputs)
    model.compile(loss =SparseCategoricalCrossentropy(), optimizer =Adam(), metrics=['accuracy'])#,SparseTopKCategoricalAccuracy(10,name='sparse_top_10'),SparseTopKCategoricalAccuracy(20,name='sparse_top_20'),SparseTopKCategoricalAccuracy(50,name='sparse_top_50')])
    model.summary()
    return model


In [6]:

tf.keras.backend.clear_session()
model = nn_model()
epoch = 150
try:
    tf.keras.models.load_model(model,'/kaggle/input/asl-signs-naidu/trained_keras')
    tf.keras.models.save_model(model,'/kaggle/working/trained_keras')
except:    
    #pass
    history = model.fit(X_train,y_train,validation_data = (X_cv,y_cv),epochs=epoch,batch_size=64,callbacks=[tf.keras.callbacks.ModelCheckpoint('v22.h5'),tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.8,patience=2,verbose=1),tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=10,verbose=1,restore_best_weights=True)])#,tf.keras.callbacks.EarlyStopping( monitor="accuracy", patience=4,verbose=1,mode="max")])
    tf.keras.models.save_model(model,'/kaggle/working/trained_keras')
    #tf.saved_model.save(model, '/kaggle/working/trained')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputty (InputLayer)           [(None, 543, 3)]     0           []                               
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 468, 3)      0           ['inputty[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.__operators__.getitem_1 (Sl  (None, 21, 3)       0           ['inputty[0][0]']                
 icingOpLambda)                                                                                   
                                                                                              

plt.figure(figsize=(15,9))
plt.plot(np.arange(1,epoch+1),history.history['loss'],color='blue',label='training_data')
plt.plot(np.arange(1,epoch+1),history.history['val_loss'],color='red',label='validation_data')
plt.legend()
plt.show()


plt.figure(figsize=(15,9))
plt.plot(np.arange(1,epoch+1),history.history['accuracy'],color='blue',label='training_data')
plt.plot(np.arange(1,epoch+1),history.history['val_accuracy'],color='red',label='validation_data')
plt.legend()
plt.show()


In [7]:
def sub_model():
    model = tf.keras.models.load_model('/kaggle/working/trained_keras')
    inputs = Input(shape=(543,3),name='inputs')
    X = tf.where(tf.math.is_nan(inputs),tf.zeros_like(inputs),inputs)
    
    X = tf.math.reduce_mean(X, axis=0,keepdims=True)
    X = model(X)
    outputs= tf.keras.layers.Activation(activation="linear",name='outputs')(X)
    infer_model = Model(inputs, outputs)
    infer_model.compile(loss =SparseCategoricalCrossentropy(), optimizer =Adam(), metrics=['accuracy',SparseTopKCategoricalAccuracy(10,name='sparse_top_10'),SparseTopKCategoricalAccuracy(20,name='sparse_top_20'),SparseTopKCategoricalAccuracy(50,name='sparse_top_50')])
    infer_model.summary()
    return infer_model

tf.keras.backend.clear_session()
infer_model= sub_model()
try :
    tf.saved_model.load('/kaggle/input/asl-signs-naidu/infer_model')
    tf.saved_model.save(infer_model, '/kaggle/working/infer_model')
except:
    tf.saved_model.save(infer_model, '/kaggle/working/infer_model')


# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('/kaggle/working/infer_model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)



!zip submission.zip $"model.tflite"    

interpreter = tflite.Interpreter('model.tflite')
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")
for i in tqdm.tqdm(range(100)):
    frames = load_relevant_data_subset(f'/kaggle/input/asl-signs/{train.iloc[i].path}')
    output = prediction_fn(inputs=frames)
    if i < 100:
        sign = np.argmax(output["outputs"])
        print(f"Predicted label: {prediction_to_sign_map[sign]}, Actual Label: {train.iloc[i].sign}")


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, 543, 3)]     0           []                               
                                                                                                  
 tf.math.is_nan (TFOpLambda)    (None, 543, 3)       0           ['inputs[0][0]']                 
                                                                                                  
 tf.zeros_like (TFOpLambda)     (None, 543, 3)       0           ['inputs[0][0]']                 
                                                                                                  
 tf.where (TFOpLambda)          (None, 543, 3)       0           ['tf.math.is_nan[0][0]',         
                                                                  'tf.zeros_like[0][0]',      

  3%|▎         | 3/100 [00:00<00:08, 10.90it/s]

Predicted label: blow, Actual Label: blow
Predicted label: happy, Actual Label: wait
Predicted label: cloud, Actual Label: cloud
Predicted label: bird, Actual Label: bird


  7%|▋         | 7/100 [00:00<00:05, 15.99it/s]

Predicted label: owie, Actual Label: owie
Predicted label: duck, Actual Label: duck
Predicted label: minemy, Actual Label: minemy
Predicted label: lips, Actual Label: lips


...

Predicted label: scissors, Actual Label: scissors
Predicted label: pajamas, Actual Label: now
Predicted label: TV, Actual Label: TV
Predicted label: awake, Actual Label: wake
Predicted label: owl, Actual Label: man
Predicted label: sticky, Actual Label: sticky
Predicted label: ride, Actual Label: jump


 94%|█████████▍| 94/100 [00:05<00:00, 30.15it/s]

Predicted label: sleep, Actual Label: sleep
Predicted label: sun, Actual Label: sun
Predicted label: first, Actual Label: first
Predicted label: yellow, Actual Label: yellow
Predicted label: fireman, Actual Label: brother
Predicted label: grass, Actual Label: grass


100%|██████████| 100/100 [00:05<00:00, 18.30it/s]

Predicted label: uncle, Actual Label: uncle
Predicted label: flag, Actual Label: fish
Predicted label: lamp, Actual Label: scissors
Predicted label: carrot, Actual Label: cowboy
Predicted label: snow, Actual Label: snow
